In [21]:
%%time
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE, SVMSMOTE
from collections import Counter
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score, cross_val_predict,StratifiedShuffleSplit, StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.feature_selection import SelectPercentile, chi2

CPU times: user 125 µs, sys: 8 µs, total: 133 µs
Wall time: 137 µs


In [23]:
%%time
with open('/content/drive/MyDrive/Colab Notebooks/train.txt','r') as train:
  train=train.readlines()
  train_values=[]
  train_labels=[]

  for i in train:
    columns=[0 for i in range(100000)]
    train_labels.append(int(i[0]))
    i=i.replace("0\t","")
    i=i.replace("1\t","")
    i=i.replace("\n","")
    new_list=list(map(int,i.split(" ")[:-1]))
    for i in new_list:
      columns[i-1]=1
    train_values.append(columns)
  x=np.array(train_values)
  y=np.array(train_labels)

CPU times: user 7.2 s, sys: 79.6 ms, total: 7.28 s
Wall time: 7.28 s


In [24]:
%%time
with open('/content/drive/MyDrive/Colab Notebooks/test.txt','r') as test:
  test=test.readlines()
  test_values=[]

  for i in test:
    columns=[0 for i in range(100000)]
    new_list=list(map(int,i.split(" ")[:-1]))
    for i in new_list:
      columns[i-1]=1
    test_values.append(columns)
  test=np.array(test_values)

CPU times: user 2.94 s, sys: 122 ms, total: 3.07 s
Wall time: 3.08 s


In [25]:
%%time
shuffle = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_records, test_records in shuffle.split(x, y):
  X_train, X_test = x[train_records], x[test_records]
  y_train, y_test = y[train_records], y[test_records]

CPU times: user 61.2 ms, sys: 2 µs, total: 61.2 ms
Wall time: 63.6 ms


In [26]:
%%time
def adaboost(X, y, n_models, random_state=0):
  y_sign = np.where(y==0, -1, 1)
  sample_weight = np.ones(len(X))*(1/len(X))
  models_ = []
  alphas = []
  weights = []
  var = 0
  for i in range(n_models):
    seed = np.random.RandomState(random_state)
    model = DecisionTreeClassifier(max_depth=1, random_state=seed)
    model.fit(X, y_sign, sample_weight=sample_weight)
    models_.append(model)
    y_prediction = model.predict(X)
    incorrects = (y_prediction != y_sign).astype(int)
    e = np.sum(incorrects*sample_weight)
    performance = 0.5 * np.log((1-e)/e)
    alphas.append(performance)
    if i!= (n_models-1):
      sample_weight *= np.exp(-1*performance*y_sign*y_prediction)
      sample_weight /= sample_weight.sum()
    var += performance*y_prediction
  result = np.where(np.sign(var)==-1, 0, 1)
  return result, alphas, models_

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 7.87 µs


In [27]:
%%time
def predict_values(X, alphas, models):
  value = 0
  for index, est in enumerate(models):
    y_pred = est.predict(X)
    value += alphas[index]*y_pred
  result = np.where(np.sign(value)==-1, 0, 1)
  return result

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.06 µs


In [28]:
%%time
def cross_validation_score(n_splits, X, y,alphas,estimators):
  cv = StratifiedKFold(n_splits)
  output = []
  for train_loc, test_loc in cv.split(X, y):
    X_train, X_test = X[train_loc], X[test_loc]
    y_train, y_test = y[train_loc], y[test_loc]
    y_prediction = predict_values(X_test,alphas,estimators)
    output.append(np.round(f1_score(y_test, y_prediction), 3))
  return output

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.63 µs


In [31]:
%%time
for i in range(100,501,100):
  reduction = SelectPercentile(chi2, percentile=0.1)
  new_x_train = reduction.fit_transform(X_train, y_train)
  sm = SVMSMOTE()
  x_smote, y_smote = sm.fit_resample(new_x_train, y_train)
  yp_train, alphas, estimators = adaboost(x_smote, y_smote, i, random_state=42)
  scores = cross_validation_score(5, x_smote, y_smote,alphas,estimators)
  print(f'Number of estimators: {i}')
  print('Mean of f1 Score: ', np.round(np.mean(scores), 3))
  print('variance of f1 score: ', np.round(np.std(scores), 3))
  print("-----------------------------------------------------")

Number of estimators: 100
Mean of f1 Score:  0.813
variance of f1 score:  0.193
-----------------------------------------------------
Number of estimators: 200
Mean of f1 Score:  0.843
variance of f1 score:  0.184
-----------------------------------------------------
Number of estimators: 300
Mean of f1 Score:  0.792
variance of f1 score:  0.17
-----------------------------------------------------
Number of estimators: 400
Mean of f1 Score:  0.753
variance of f1 score:  0.172
-----------------------------------------------------
Number of estimators: 500
Mean of f1 Score:  0.758
variance of f1 score:  0.148
-----------------------------------------------------
CPU times: user 6.67 s, sys: 509 ms, total: 7.17 s
Wall time: 6.63 s


In [32]:
for i in range(10,101,10):
  reduction = SelectPercentile(chi2, percentile=i/100)
  new_x_train = reduction.fit_transform(X_train, y_train)
  sm = SVMSMOTE()
  x_smote, y_smote = sm.fit_resample(new_x_train, y_train)
  yp_train, alphas, estimators = adaboost(x_smote, y_smote, 100, random_state=42)
  scores = cross_validation_score(5, x_smote, y_smote,alphas,estimators)
  print(f'percentile : {i}')
  print('Mean of f1 Score: ', np.round(np.mean(scores), 3))
  print('variance of f1 score: ', np.round(np.std(scores), 3))
  print("-----------------------------------------------------")

percentile : 10
Mean of f1 Score:  0.812
variance of f1 score:  0.205
-----------------------------------------------------
percentile : 20
Mean of f1 Score:  0.722
variance of f1 score:  0.203
-----------------------------------------------------
percentile : 30
Mean of f1 Score:  0.863
variance of f1 score:  0.151
-----------------------------------------------------
percentile : 40
Mean of f1 Score:  0.858
variance of f1 score:  0.121
-----------------------------------------------------
percentile : 50
Mean of f1 Score:  0.837
variance of f1 score:  0.121
-----------------------------------------------------
percentile : 60
Mean of f1 Score:  0.813
variance of f1 score:  0.121
-----------------------------------------------------
percentile : 70
Mean of f1 Score:  0.83
variance of f1 score:  0.129
-----------------------------------------------------
percentile : 80
Mean of f1 Score:  0.89
variance of f1 score:  0.093
-----------------------------------------------------
percentile

In [33]:
%%time
reduction = SelectPercentile(chi2, percentile=0.1)
new_x_train = reduction.fit_transform(X_train, y_train)
sm = SVMSMOTE()
x_smote, y_smote = sm.fit_resample(new_x_train, y_train)
yp_train, alphas, estimators = adaboost(x_smote, y_smote, 100, random_state=42)
new_x_test = reduction.transform(X_test)
new_x_test, y_test_smote = sm.fit_resample(new_x_test, y_test)
yp_test = predict_values(new_x_test, alphas, estimators)
print(f'F1 score of test set: {f1_score(y_test_smote, yp_test)}')

F1 score of test set: 0.8294573643410853
CPU times: user 957 ms, sys: 106 ms, total: 1.06 s
Wall time: 945 ms


In [34]:
%%time
new_test = reduction.transform(test)
test_out = predict_values(new_test,alphas, estimators)
file = open('/content/drive/MyDrive/Colab Notebooks/result.txt', 'w')
for i in test_out:
  file.write(str(i))
  file.write("\n")
file.close()

CPU times: user 16.3 ms, sys: 144 µs, total: 16.4 ms
Wall time: 19.2 ms
